# 学習システムでのMNISTの学習とモデル・重みの保存

---

学習システムで、MNISTデータを学習するために必要なスクリプト等の転送、MNISTデータの学習、学習モデルと学習結果としての重みを保存し、フロントエンドに転送します。なお、本Notebookでは VC ノード上で動作している Slurm や TensorFlow にアクセスするのみのため、VCP は使用しません。

MNIST データの学習は、[OpenHPC-v2](https://github.com/nii-gakunin-cloud/ocs-templates/tree/master/OpenHPC-v2) `152-NGCのコンテナ実行-TensorFlow.ipynb`と基本的に同じですが、学習したモデルと重みを保存する点が異なります。

![構成](images/220.training.png)

## 学習システムの再開

学習システムを停止している場合は、`1912-ノードの再開.ipynb`を使用し学習システムを再開してください。
(学習システムの停止は`1911-ノードの停止.ipynb`を使用して実施できます)

## 認識システムの指定

認識システムと関連する学習システムの情報を読み出すため、認識システムの `UnitGroup` 名を指定します。なお、内部的には UnitGroup 名のついたファイル名をアクセスするだけで、VCP へはアクセスしません。

In [ ]:
rs_ugroup_name = 'TfCpu'

## 学習システムでの MNIST データの学習と結果の保存

### 準備

#### 認識、学習システムのパラメータ読み出し

認識システムと関連する学習システムの情報を読み出します。以下の場合エラーとなります。
* 認識システム構築時に学習システムの使用を定義していない（training_system = False)
* 作業用フォルダ(../1.learning_system/{ohpc_WORK_DIR})が存在しない
* 指定した UnitGroup ({ts_ugroup_name})が存在しない

In [ ]:
%run scripts/ts/group.py

rs_gvars = load_group_vars(rs_ugroup_name)
print(rs_gvars)

if rs_gvars['training_system']:
    ts_gvars = load_group_vars(rs_gvars['ts_ugroup_name'], "../1.learning_system/" + rs_gvars['ohpc_WORK_DIR'])
    print(ts_gvars)
else:
    raise RuntimeError('Error: no training systems')

#### 固定パラメタの設定

In [ ]:
# ユーザ名とホスト名
ts_target = f'{ts_gvars["ohpc_user"]}@{ts_gvars["master_ipaddress"]}'

# 学習システムと認識システムの TensorFlow 作業フォルダ（共通パス名）
tf_work_dir = rs_gvars['tf_work_dir']

# SSHのコマンドライン引数
ts_user_ssh_opts = f'-i {ts_gvars["ohpc_user_prvkey"]}'

# 使用するノード数
nodes = 1

# 認識システムの IP アドレス
rs_ipaddress = rs_gvars['rs_ipaddress']
print(rs_ipaddress)

#### 学習システム用のスクリプトの準備と転送

ジョブスクリプトのテンプレートに、上のセルで定義した user, work_dir を適用したジョブスクリプトを生成します。

In [ ]:
import shutil
import os
org_path='./original/job'
job_path='./scripts/job'
if os.path.isdir(job_path):
    shutil.rmtree(job_path)
    
os.makedirs(job_path)
for fname in os.listdir(org_path):
    !sed 's/__xUSERx__/{rs_gvars["rs_user"]}/g' {org_path}/{fname} | sed 's/__xWORK_DIRx__/{tf_work_dir}/g' > {job_path}/{fname}

学習スクリプトとジョブスクリプトを学習システムへ転送します。MNIST の学習スクリプトは、[OpenHPC-v2](https://github.com/nii-gakunin-cloud/ocs-templates/tree/master/OpenHPC-v2) テンプレートに添付されている学習スクリプトにモデルと重みの保存を追加したものです。

In [ ]:
# 学習システムに、データやスクリプトを配置するディレクトリを作成
!ssh {ts_user_ssh_opts} {ts_target} mkdir -p {tf_work_dir}

# 学習スクリプトとジョブスクリプトの転送
!scp {ts_user_ssh_opts} scripts/tensorflow/*.py {ts_target}:{tf_work_dir}
!scp {ts_user_ssh_opts} scripts/job/*.job {ts_target}:{tf_work_dir}

# コンテナを実行するため、ホームディレクトリのパーミッションを変更
!ssh {ts_user_ssh_opts} {ts_target} chmod 755 .

#### バッチジョブ操作関数の定義

In [ ]:
# バッチジョブ投入関数定義(slurm 用)
def qsub(job, work_dir="."):
    bs_state = !ssh {ts_user_ssh_opts} {ts_target} bash -l -c "'cd {work_dir} && sbatch {job}'"
    bs_state = bs_state[len(bs_state)-1]
    print(bs_state)

    if bs_state.split()[0] == 'Submitted':
        return bs_state.split()[3]
    else:
        raise RuntimeError('ERROR: job submission error')

        
def qstat(bs_jobid):
    print('job ' + bs_jobid + ':', end = ' ')
    while True:
        bs_state = !ssh {ts_user_ssh_opts} {ts_target} bash -l -c 'squeue --noheader --jobs={bs_jobid}'
        bs_state = bs_state[len(bs_state)-1]
        if bs_state == '' or bs_state.split()[0] == 'JOBID':
            break
        elif 'error' in bs_state:
            raise RuntimeError(bs_state)
        else:
            print("", end = '.')
            !sleep 5
    print(' finished')

### MNIST データの学習

MNIST の学習ジョブ(tensorflow_mnist_docker_ho.job)を実行します。この学習ジョブスクリプトでは、学習スクリプト mnist_training.py を実行します。終了までに数分かかります。ジョブ実行中のCPU, メモリ, GPUなどの利用状況は VCC の Grafana で確認することもできます。

In [ ]:
# 以前の学習ジョブ結果の削除
!ssh {ts_user_ssh_opts} {ts_target} bash -l -c \
    "'cd {tf_work_dir} && rm -rf  tensorflow-mnist*.out saved_model*'"

# 学習ジョブの投入と実行終了待ち
jobid = qsub('tensorflow_mnist_docker_ho.job', tf_work_dir)
qstat(jobid)

ジョブの出力結果を確認します。

In [ ]:
!ssh {ts_user_ssh_opts} {ts_target} bash -l -c \
    "'cd {tf_work_dir} && tail tensorflow-mnist*.out'"

ジョブの実行が完了していれば出力結果の末尾に以下のような内容が表示されます。

```
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1098 - accuracy: 0.9664
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0891 - accuracy: 0.9724
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0748 - accuracy: 0.9755
313/313 - 0s - loss: 0.0839 - accuracy: 0.9748

Test accuracy: 0.9747999906539917
```

### モデルと重みの転送
学習システムで学習したモデルと重みを、認識システムに展開します。

In [ ]:
# 学習システムでの学習結果をフロントエンドにダウンロード
!scp {ts_user_ssh_opts} -p {ts_target}:{tf_work_dir}/saved_model*.tgz {rs_gvars['fe_save_dir']}